# MQTT-Protokoll

Message Queuing Telemetry Transport (MQTT) ist ein Netzwerkprotokoll, welches Daten in Form von Nachrichten zwischen Geräten (z.B. Sensoren, Aktoren, Steuergeräten, Datenservern) übermittelt.

MQTT arbeitet als Client-Server-Protokoll, wobei der Client an den Server (“Broker”) nach erfolgreichem Verbindungsaufbau Nachrichten mit einem eindeutig bezeichneten Topic sendet ("publishen"). Die Bezeichnung der Topics erfolgt hierbei eindeutig und hierarchisch, z.B. DHT22/Temperatur oder DHT22/Feuchte. 

Clients können diese gekennzeichneten Nachrichten vom Broker auch abonnieren ("subscriben"), welcher die empfangenen Nachrichten an die entsprechenden Abonnenten weiterleitet. Die Nachrichten bestehen aus dem Topic und dem Nachrichteninhalt, z.B. der von einem Sensor erfassten und dann veröffentlichten Messwert. 

<img src="./img/MQTT_protocol_example.png" width="600">

Quelle: https://en.wikipedia.org/wiki/MQTT

Aufbau der Verbindung zum NodeMCU über die serielle Schnittstelle des PC

In [ ]:
%serialconnect

## Aufbau einer WLAN-Verbindung von NodeMCU zum lokalen Netzwerk

In [ ]:
from umqtt.simple import MQTTClient

SSID = 'ssid'
PASSWORD = 'password'

def do_connect():
    import network
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    if not wlan.isconnected():
        print('connecting to network...')
        wlan.connect(SSID, PASSWORD)
        while not wlan.isconnected():
            pass
    print('network config:', wlan.ifconfig())

do_connect()

## Instantiieren eines MQTT-Clients

In [ ]:
client=MQTTClient(client_id= 'DHT22',server= 'test.mosquitto.org', port= 1883, keepalive=30)

## Initialisieren des NodeMCU auf UTC-Zeit

Die UTC (Coordinated Universal Time) ist die koordinierte Weltzeit, welche 1972 eingeführt wurde. Man erhält die Mitteleuropäische Zeit (MEZ), indem man eine Stunde bzw. die Mitteleuropäische Sommerzeit (MESZ), indem man zwei Stunden zur UTC addiert.

Die Real Time Clock der NodeMCU beginnt mit ihrer Zeitrechnung am 01.01.2000. Da sonst üblicherweise ab dem 01.01.1970 gezählt muss man bei der Übergabe von Timestamps von der NodeMCU an Linux Windows oder Mac-Systeme 946767600 Sekunden dazurechnen.

In [ ]:
from machine import RTC
import ntptime

rtc = RTC()
ntptime.settime() # set the rtc datetime from the remote server
print(rtc.datetime()) # get the date and time in UTC


## Veröffentlichen (publishen) von Informationen an den MQTT-Broker

Veröffentlichen der Messwerte über separate Topics

In [ ]:
import time
from machine import Pin
import dht 

sensor = dht.DHT22(Pin(4))

while True:
  try:
    client.connect()
    sensor.measure()
    temp = sensor.temperature()
    hum = sensor.humidity()
    time.sleep(1)
    client.publish('Coding_Seminar/DHT22/time',str(time.time() + 946767600)) # Offset für die Verarbeitung auf Linux/Windows/Mac (s.o.)
    client.publish('Coding_Seminar/DHT22/temp',str(temp))
    client.publish('Coding_Seminar/DHT22/hum',str(hum))
    time.sleep(1)
    client.disconnect()
  except OSError as e:
    print('Failed to read sensor.')

Veröffentlichen aller Messwerte über einen Topic

In [ ]:
import time
from machine import Pin
import dht 

sensor = dht.DHT22(Pin(4))
while True:
  try:
    client.connect()
    sensor.measure()
    timestamp = time.time() + 946767600 # Offset für die Verarbeitung auf Linux/Windows/Mac (s.o.)
    temp = sensor.temperature()
    hum = sensor.humidity()
    data = (timestamp,temp,hum)
    time.sleep(1)
    client.publish('Coding_Seminar/DHT22',str(data))
    time.sleep(1)
    client.disconnect()
  except OSError as e:
    print('Failed to read sensor.')